# Stateful LSTM for a One-Char to One-Char Mapping

We have seen that we can break-up our raw data into fixed size sequences and that this representation can be learned by the LSTM, but only to learn random mappings of 3 characters to 1 character.

We have also seen that we can pervert batch size to offer more sequence to the network, but only during training.

Ideally, we want to expose the network to the entire sequence and let it learn the inter-dependencies, rather than us define those dependencies explicitly in the framing of the problem.

We can do this in Keras by making the LSTM layers stateful and manually resetting the state of the network at the end of the epoch, which is also the end of the training sequence.

This is truly how the LSTM networks are intended to be used. We find that by allowing the network itself to learn the dependencies between the characters, that we need a smaller network (half the number of units) and fewer training epochs (almost half).

In [17]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences

# fix random seed for reproducibility
numpy.random.seed(7)

# define the raw dataset
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))

# prepare the dataset of input to output pairs encoded as integers
seq_length = 1
dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
    seq_in = alphabet[i:i + seq_length]
    seq_out = alphabet[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    print(seq_in, '->', seq_out)

A -> B
B -> C
C -> D
D -> E
E -> F
F -> G
G -> H
H -> I
I -> J
J -> K
K -> L
L -> M
M -> N
N -> O
O -> P
P -> Q
Q -> R
R -> S
S -> T
T -> U
U -> V
V -> W
W -> X
X -> Y
Y -> Z


In [28]:
# convert list of lists to array and pad sequences if needed
#X = pad_sequences(dataX, maxlen=seq_length, dtype='float32')
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (len(dataX), seq_length, 1))
# normalize
X = X / float(len(alphabet))
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [29]:
samples, time_steps, features = X.shape
print("Samples: ", samples)
print("Time Steps: ", time_steps)
print("Features: ", features)

Samples:  25
Time Steps:  1
Features:  1


We first need to define our LSTM layer as stateful. In so doing, we must explicitly specify the batch size as a dimension on the input shape. This also means that when we evaluate the network or make predictions, we must also specify and adhere to this same batch size. This is not a problem now as we are using a batch size of 1. This could introduce difficulties when making predictions, when the batch size is not one as predictions, will need to be made in batch and in sequence.

In [31]:
# create and fit the model
batch_size = 1
model = Sequential()
model.add(LSTM(16, batch_input_shape=(batch_size, time_steps, features), stateful=True))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

An important difference in training the stateful LSTM is that we train it manually one epoch at a time and reset the state after each epoch. We can do this in a for loop. Again, we do not shuffle the input, preserving the sequence in which the input training data was created.

In [32]:
for i in range(300):
    model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
    model.reset_states()

Epoch 1/1
 - 0s - loss: 3.2704 - acc: 0.0400
Epoch 1/1
 - 0s - loss: 3.2568 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.2500 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.2434 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.2368 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.2297 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.2219 - acc: 0.1200
Epoch 1/1
 - 0s - loss: 3.2129 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.2022 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.1893 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.1735 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.1542 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.1321 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.1086 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.0851 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.0628 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.0434 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.0297 - acc: 0.1200
Epoch 1/1
 - 0s - loss: 3.0254 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.0324 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.0432 - acc: 0.0800
Epoch 1/1
 - 0s - loss: 3.0440 - acc: 0.0800
Epoch 1/1


 - 0s - loss: 2.0073 - acc: 0.1600
Epoch 1/1
 - 0s - loss: 1.9891 - acc: 0.1600
Epoch 1/1
 - 0s - loss: 1.9652 - acc: 0.1600
Epoch 1/1
 - 0s - loss: 1.9367 - acc: 0.1600
Epoch 1/1
 - 0s - loss: 1.9065 - acc: 0.1600
Epoch 1/1
 - 0s - loss: 1.8803 - acc: 0.1600
Epoch 1/1
 - 0s - loss: 1.8578 - acc: 0.2000
Epoch 1/1
 - 0s - loss: 1.8350 - acc: 0.2000
Epoch 1/1
 - 0s - loss: 1.8139 - acc: 0.2000
Epoch 1/1
 - 0s - loss: 1.7950 - acc: 0.2000
Epoch 1/1
 - 0s - loss: 1.7848 - acc: 0.2000
Epoch 1/1
 - 0s - loss: 1.7736 - acc: 0.2400
Epoch 1/1
 - 0s - loss: 1.7633 - acc: 0.2800
Epoch 1/1
 - 0s - loss: 1.7541 - acc: 0.2800
Epoch 1/1
 - 0s - loss: 1.7491 - acc: 0.2800
Epoch 1/1
 - 0s - loss: 1.7530 - acc: 0.2800
Epoch 1/1
 - 0s - loss: 1.7490 - acc: 0.2800
Epoch 1/1
 - 0s - loss: 1.7442 - acc: 0.2800
Epoch 1/1
 - 0s - loss: 1.7430 - acc: 0.2400
Epoch 1/1
 - 0s - loss: 1.7389 - acc: 0.2400
Epoch 1/1
 - 0s - loss: 1.7331 - acc: 0.2400
Epoch 1/1
 - 0s - loss: 1.7237 - acc: 0.2400
Epoch 1/1
 - 0s - lo

As mentioned, we specify the batch size when evaluating the performance of the network on the entire training dataset.

In [33]:
# summarize performance of the model
scores = model.evaluate(X, y, batch_size=batch_size, verbose=0)
model.reset_states()
print("Model Accuracy: %.2f%%" % (scores[1]*100))

Model Accuracy: 40.00%


Finally, we can demonstrate that the network has indeed learned the entire alphabet. We can seed it with the first letter “A”, request a prediction, feed the prediction back in as an input, and repeat the process all the way to “Z”.

In [34]:
# demonstrate some model predictions
seed = [char_to_int[alphabet[0]]]
for i in range(0, len(alphabet)-1):
    x = numpy.reshape(seed, (1, len(seed), 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    print(int_to_char[seed[0]], "->", int_to_char[index])
    seed = [index]
model.reset_states()

A -> B
B -> C
C -> D
D -> D
D -> E
E -> G
G -> G
G -> H
H -> I
I -> J
J -> K
K -> M
M -> M
M -> Q
Q -> Q
Q -> Q
Q -> Q
Q -> R
R -> T
T -> U
U -> V
V -> W
W -> X
X -> Y
Y -> Z


We can also see if the network can make predictions starting from an arbitrary letter.

In [35]:
# demonstrate a random starting point
letter = "K"
seed = [char_to_int[letter]]
print("New start: ", letter)
for i in range(0, 5):
    x = numpy.reshape(seed, (1, len(seed), 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    print(int_to_char[seed[0]], "->", int_to_char[index])
    seed = [index]
model.reset_states()

New start:  K
K -> B
B -> C
C -> D
D -> E
E -> F


We can see that the network has memorized the entire alphabet perfectly. It used the context of the samples themselves and learned whatever dependency it needed to predict the next character in the sequence.

We can also see that if we seed the network with the first letter, that it can correctly rattle off the rest of the alphabet.

We can also see that it has only learned the full alphabet sequence and that from a cold start. When asked to predict the next letter from “K” that it predicts “B” and falls back into regurgitating the entire alphabet.

To truly predict “K” the state of the network would need to be warmed up iteratively fed the letters from “A” to “J”. This tells us that we could achieve the same effect with a “stateless” LSTM by preparing training data like:

`---a -> a`

`--ab -> c`

`-abc -> d`

`abcd -> e`

Where the input sequence is fixed at 25 (a-to-y to predict z) and patterns are prefixed with zero-padding.

Finally, this raises the question of training an LSTM network using variable length input sequences to predict the next character.